In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd 
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split

In [4]:
#Load the data into Pandas dataframe
df = pd.read_csv('../../task2_data/task2_en_training.tsv', sep='\t')[['class', 'tweet']]
df['class'] = df['class'].apply(lambda x: int(x))
df

,class,tweet
0,0,"depression hurts, cymbalta can help"
1,0,"@jessicama20045 right, but cipro can make thin..."
2,0,@fibby1123 are you on paxil .. i need help
3,0,@redicine the lamotrigine and sjs just made ch...
4,0,have decided to skip my #humira shot today. my...
...,...,...
20539,1,@hornetweb my mri scan shows when it happens b...
20540,1,remember 2003. vioxx costs $65m to ontario dru...
20541,1,asians are at higher risk for severe allergic ...
20542,1,5. so what caused the #estrogen surges in #nuv...


In [5]:
df = df.iloc[np.random.permutation(len(df))]
df.head()

,class,tweet
10335,0,even tho when i first tried vimpat i was prett...
6442,0,surely temazepam can't interact with one beer ...
14675,0,preparing for tanzania. have eaten all the old...
16636,0,#pharmacy discounts - save up to 65% - @winndi...
2238,0,@trainingwithra @rawarrior @ginarclark my enbr...


In [6]:
X = df['tweet'][:800]
Y = df['class'][:800]

X_train, X_test, y_train, y_test = train_test_split(X, Y , test_size=0.28)

tfidf = TfidfVectorizer()
tfidf.fit(X_train)
X_train = tfidf.transform(X_train)
X_test = tfidf.transform(X_test)
    
# type(X_train), type(y_train)

# X_train = Train['tweet']
# y_train = Train['type']

# X_test = Test['tweet']
# y_test = Test['type']

# tfidf = TfidfVectorizer()
# tfidf.fit(X_train)

# X_train = tfidf.transform(X_train)
# X_test = tfidf.transform(X_test)


In [7]:
X_train[1]

<1x3179 sparse matrix of type '<class 'numpy.float64'>'
	with 15 stored elements in Compressed Sparse Row format>

In [8]:
SVM = SVC(kernel='linear')
SVM.fit(X_train,y_train)

predictions_SVM = SVM.predict(X_test)

In [9]:
round(f1_score(predictions_SVM, y_test, average='macro')*100, 3)

47.674

# SHUFFLE SPLIT + Cross Val.

In [13]:
from sklearn.model_selection import ShuffleSplit, cross_val_score, cross_val_predict
from scipy.sparse import hstack

data = df
data = data.iloc[np.random.permutation(len(df))]
data

,class,tweet
17482,0,after an entire day of people telling me that ...
15399,0,"the vast amounts of methadone, ketamine &amp; ..."
6204,0,@scotthortonshow their only win was thalidomid...
8361,1,@stopthatimp geodon withdrawl is hell.
15748,0,nexium judge not giving defendants directed ve...
...,...,...
19799,0,switching from enbrel to orencia. has anyone d...
15754,0,@caseylblair w/the synthroid in the morning i’...
18091,0,hearing a talk about sildenafil and pulmonary ...
20345,1,@scfmito ooooo geeez that's not good. did they...


In [11]:
X = data['tweet']
Y = data['class']

X_train, X_test, y_train, y_test = train_test_split(X, Y , test_size=0.2)

tfidf = TfidfVectorizer()
tfidf.fit(X_train)
X_train = tfidf.transform(X_train)
X_test = tfidf.transform(X_test)
    
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(16435, 30535)
(4109, 30535)
(16435,)
(4109,)


In [12]:
SVM2 = SVC(kernel='linear', C=1) #.fit(X_train,y_train)
tols = [0.1, 0.01, 0.001, 0.0001]
best_score = 0
best_tol = 0
for tol in tols:
    SVM2 = SVC(kernel='linear', C=1, tol=tol)
    scores = cross_val_score(SVM2, X_train, y_train, cv=3, scoring='f1_macro')
    m = scores.mean()
    if m > best_score:
        best_score = m
        best_tol = tol
    print('tol =', tol, 'score =', scores.mean())
    
print('best_tol =', best_tol, 'best_score =', best_score)

tol = 0.1 score = 0.6053333750339219
tol = 0.01 score = 0.6057646001396112


KeyboardInterrupt: 

In [ ]:
print(classification_report(y_test, predictions_SVM))